## Capstone Project : Britties Yoga Snack
 
## PART I 

### Problem and Backgound

"Britties" is a new snack, created and developed for the yoga community. It was created by three students who were studying business at New York University (NYU) and wanted to start their entrepreneurial life. One of them was also a yoga professor, who saw a need of a new snack for the yoga community. The product they created  has the best taste and amount of protein per snack and they claimed that it would be a hit their product. In order validate their idea , the want to launch a pilot in the NY area, but due the the limitation in their budget, they must focus in just one area of NY to do the pilot, so they want to apply their data science knowledge in order to find the proper area . 

### Stakeholders

The main stakeholders would be the NY citizens, the yoga community and the three fellow students

## PART II

### Methodology

In order to make the decision differents factors would need to be considered. The first one would be the borough who will have the most Yoga Studio and in second place which Area in the borough will have more density of yoga places. 

### Working with the data

The data that would be used for the project will be the NY Map with the neighbour´s name, latitude, longitude, among other things,  and the data of the yoga places that will be segmented in Foursquare by the category ID:4bf58dd8d48988d102941735

## PART III 

### Processing the Data

#### 1-Importing the libraries

In [25]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
import geopy

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


#### 2-Importing the data

In [2]:
import urllib.request
url = 'https://cocl.us/new_york_dataset'
filename = 'newyork_data.json'
urllib.request.urlretrieve(url, filename)
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as NYDF:
    NYDF = json.load(NYDF)

In [4]:
NYDF

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

#### 3-Transforming the data

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# Transform dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [7]:
neighborhoods_data = NYDF['features']

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    


In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### 4-Visualizing the data in a Map

In [10]:
# Use geopy library to get the latitude and longitude values of New York City.
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [11]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
folium.TileLayer('cartodbdark_matter').add_to(map_newyork)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#fbffb5',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### 5-Finding the Yoga Places with Foursquare

In [12]:
CLIENT_ID = 'XLNOKOC5VOWK2HJFY1MHYXWSJDBKUXGIIX4BJSO2O3BMLPI5' # your Foursquare ID
CLIENT_SECRET = 'HBJIKGWE1XZM0NUMIQE33M5H4JHVAHDXQQUZJ1IAVCZ0XCY1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XLNOKOC5VOWK2HJFY1MHYXWSJDBKUXGIIX4BJSO2O3BMLPI5
CLIENT_SECRET:HBJIKGWE1XZM0NUMIQE33M5H4JHVAHDXQQUZJ1IAVCZ0XCY1


In [14]:
LIMIT = 500 #maximum venues with the current account
boroughs = ["Bronx","Manhattan","Brooklyn","Queens","Staten Island"]
results = {}
for borough in boroughs:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        borough,
        LIMIT,
        "4bf58dd8d48988d102941735") # YOGA STUDIO ID
    results[borough] = requests.get(url).json()

In [15]:
results

{'Bronx': {'meta': {'code': 200, 'requestId': '5ec7b2ad3907e70028484f6f'},
  'response': {'geocode': {'what': '',
    'where': 'bronx',
    'center': {'lat': 40.84985, 'lng': -73.86641},
    'displayString': 'Bronx, NY, United States',
    'cc': 'US',
    'geometry': {'bounds': {'ne': {'lat': 40.917577, 'lng': -73.74806},
      'sw': {'lat': 40.785743, 'lng': -73.933808}}},
    'slug': 'bronx-new-york',
    'longId': '72057594043038202'},
   'headerLocation': 'Bronx',
   'headerFullLocation': 'Bronx',
   'headerLocationGranularity': 'city',
   'query': 'yoga studio',
   'totalResults': 9,
   'suggestedBounds': {'ne': {'lat': 40.8881664, 'lng': -73.82627965},
    'sw': {'lat': 40.8248416, 'lng': -73.92898335000001}},
   'groups': [{'type': 'Recommended Places',
     'name': 'recommended',
     'items': [{'reasons': {'count': 0,
        'items': [{'summary': 'This spot is popular',
          'type': 'general',
          'reasonName': 'globalInteractionReason'}]},
       'venue': {'id': '

#### 6-Getting the Data

In [21]:
df_yoga={}
for borough in boroughs:
    venues = pd.json_normalize(results[borough]['response']['groups'][0]['items'])
    df_yoga[borough] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_yoga[borough].columns = ['Name', 'Address', 'Lat', 'Lng']

In [22]:
df_yoga

{'Bronx':                               Name                    Address        Lat  \
 0                      Bikram Yoga              5500 Broadway  40.876844   
 1                 One Yoga For All           3264 Johnson Ave  40.885288   
 2                       Excel Yoga             3045 Buhre Ave  40.847806   
 3  Sweet Water Dance & Yoga Studio             876 Gerard Ave  40.827720   
 4             Zen-Fit Rehab Studio             410 E 173rd St  40.842716   
 5           Bronx Dahn Yoga Center     1805 Williamsbridge Rd  40.850340   
 6                    B-Well Studio     2125 Williamsbridge Rd  40.856036   
 7                      Zambo Aroma       3848 White Plains Rd  40.885104   
 8                      FinerStudio  1710 Bronxdale Ave Apt 2E  40.845791   
 
          Lng  
 0 -73.906204  
 1 -73.909588  
 2 -73.830948  
 3 -73.924315  
 4 -73.903150  
 5 -73.851810  
 6 -73.855978  
 7 -73.861626  
 8 -73.858552  ,
 'Manhattan':                                             

## Conclusion Part I Capstone Project

#### Now that the we have the data with the yoga places, in the second part of the project ( for the next week ), we will analyze more deeper the locations, in order to find the proper place for the pilot